# Architektur Neuronales Netz

In [1]:
# aNN Architektur

# Importe / Bibliotheken
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


#### Default Datentyp

In [2]:
torch.set_default_dtype(torch.float64)

#### Erzeugnung des Moduls

In [3]:
class NeuralNetwork(nn.Module):
    
    #Initalisierung der Netzwerk layers
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
    
        super().__init__() #Referenz zur Base Class (nn.Module)
        #Kaskade der Layer
        self.linear_afunc_stack = nn.Sequential(
            nn.Linear(input_size, hidden1_size), #Lineare Transformation mit gespeicherten weights und biases
            nn.Tanh(), #Nicht lineare Aktivierungsfunktion um komplexe nichtlineare Zusammenhänge abzubilden
            nn.Linear(hidden1_size, hidden2_size),
            nn.Tanh(),
            nn.Linear(hidden2_size, output_size),
        )

    #Implementierung der Operationen auf Input Daten
    def forward(self, x):
        #x = self.flatten(x)
        out = self.linear_afunc_stack(x)
        return out

#### Ausgabe Modul

In [4]:
model = NeuralNetwork(5, 20, 20, 1)
print(model)

NeuralNetwork(
  (linear_afunc_stack): Sequential(
    (0): Linear(in_features=5, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)


#### DataLoader

In [5]:
batch_size = 32 #Zahl der Datenpaare die vor einem erneuten Update der Parameter ins Netzt gegeben werden
eq_data_file = Path.cwd() / 'data' / 'eq_dataset.npz' #Import der GGW Daten

res = np.load(eq_data_file)

# Bei Speicherung wurden Daten als T, p, x_0 und xi gespeichert
# Inputs T, p, x_0[H2,N2,NH3]
# Outputs xi
# Umwandlen der np.arrays in torch.tensors zur besseren Arbeit mit PyTorch
T = torch.tensor(res['T'])
p = torch.tensor(res['p'])
x_0 = torch.tensor(res['x_0'])
xi = torch.tensor(res['xi'])

# T = torch.tensor(res['T']).float()
# p = torch.tensor(res['p']).float()
# x_0 = torch.tensor(res['x_0']).float()
# xi = torch.tensor(res['xi']).float()

# print(T.dtype)
# print(xi.dtype)

x_input = torch.stack((T,p,x_0[:,0],x_0[:,1],x_0[:,2]),1)
y_output = xi
#print(x_input.size())
# print(xi.size())

# Tensoren zu einem großen Set gruppieren
dataset = TensorDataset(x_input, y_output)

# for x,y in dataset:
#     print(x)
#     print(y)
    
# Split in Trainings und Test Set
train_dataset, test_dataset = random_split(dataset, 
                                           [int(0.8*len(dataset)), int(0.2*len(dataset))], # splitting 80/20
                                           generator = torch.Generator().manual_seed(42)) # Festlegung seed zur Reproduktivität

# Erzeugen der DataLoader zur Arbeit mit Daten
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True) # shuffle batches zur Reduzierung von overfitting
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)


#### Generierung Netzwerk, Festlegung von loss Funktion und Optimizer

In [6]:
# Erzeugung aNN
net = NeuralNetwork(5, 20, 20, 1)

# Loss Funktion; gibt Fehler an
loss_fn = nn.MSELoss()

#Definition custom loss Funktion, MRE
def MRELoss(outputs, targets):
    
    loss = torch.mean(abs((outputs - targets) / targets))
    
    return loss
    

#Optimizer
learning_rate = 0.001
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)


#### Funktion zur Bestimmung der Genauigkeit

In [7]:
def check_accuracy(loader, net):
    
    loss = 0
    train_correct = 0
    train_total = len(loader.dataset)
    num_batches = len(loader) 
    #train_total = 0
    
    net.eval() # Put network in evaluation mode
    
    if loader == train_dataloader:
        dataset = "Train"
    else:
        dataset = "Test"
    
    with torch.no_grad():
        for X, y in loader:
            pred = net(X)
           
            #loss = loss_fn(pred, y) # Calculate the loss
            loss += MRELoss(pred, y).item()

            # Record the correct predictions for training data
            _, predictions = torch.max(pred.data, 1)
            #train_correct += (predictions == y.data).sum()
            train_correct += (abs(predictions - y.data) <= 0.001).sum()
            #train_total += predictions.size(0)
            
        # Genauigkeit berechnen
        acc = float(train_correct) / float(train_total) * 100
        acc = round(acc, 2)
        
        loss /= num_batches
        print(f"{dataset} Error: \n Accuracy: {acc}%, Avg loss: {loss:>8f} \n")

    net.train()
    
    return acc, loss

#### Training

In [8]:
num_epochs = 500 #Iterationen über Datenset

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

#Optimierungsloop
for epoch in range(num_epochs):
    train_correct = 0
    train_total = 0
        
    for batch, (X,y) in enumerate(train_dataloader):
        
#         print(X.shape)
#         print(X.dtype)
        
        net.train() #Trainingmodus
        
        # forward
        pred = net(X)  # Do the forward pass
        #loss = loss_fn(pred, y) # Calculate the loss
        loss = MRELoss(pred, y)
        
        # backward
        optimizer.zero_grad() # Clear off the gradients from any past operation
        loss.backward()       # Calculate the gradients with help of back propagation, updating weights and biases
        
        # adam step gradient descent
        optimizer.step()      # Ask the optimizer to adjust the parameters based on the gradients
        
#         # Record the correct predictions for training data
#         _, predictions = torch.max(pred.data, 1)
#         train_correct += (predictions == y.data).sum()                
#         train_total += predictions.size(0)    

        print ('Epoch %d/%d, Iteration %d/%d, Loss: %.4f' 
               %(epoch+1, num_epochs, batch+1, len(train_dataset)//batch_size, loss.item()))
        
    
    net.eval() # Put the network into evaluation mode
    
    # Book keeping
    #train_loss.append(loss.item())
    
    # What was our train accuracy?
    train_acc, train_lost = check_accuracy(train_dataloader, net)
    
    #Record loss and accuracy
    train_accuracy.append(train_acc)
    train_loss.append(train_lost)
    
    # How did we do on the test set (the unseen set)
    # Record the correct predictions for test data
    test_acc, test_lost = check_accuracy(test_dataloader, net)
    test_accuracy.append(test_acc)
    test_loss.append(test_lost)   

#     loss = criterion(outputs, Variable(test_classes))
#     test_loss.append(loss.data[0])

Epoch 1/500, Iteration 1/25, Loss: 0.0241
Epoch 1/500, Iteration 2/25, Loss: 0.0199
Epoch 1/500, Iteration 3/25, Loss: 0.0168
Epoch 1/500, Iteration 4/25, Loss: 0.0200
Epoch 1/500, Iteration 5/25, Loss: 0.0206
Epoch 1/500, Iteration 6/25, Loss: 0.0221
Epoch 1/500, Iteration 7/25, Loss: 0.0190
Epoch 1/500, Iteration 8/25, Loss: 0.0165
Epoch 1/500, Iteration 9/25, Loss: 0.0163
Epoch 1/500, Iteration 10/25, Loss: 0.0171
Epoch 1/500, Iteration 11/25, Loss: 0.0169
Epoch 1/500, Iteration 12/25, Loss: 0.0127
Epoch 1/500, Iteration 13/25, Loss: 0.0141
Epoch 1/500, Iteration 14/25, Loss: 0.0115
Epoch 1/500, Iteration 15/25, Loss: 0.0134
Epoch 1/500, Iteration 16/25, Loss: 0.0126
Epoch 1/500, Iteration 17/25, Loss: 0.0124
Epoch 1/500, Iteration 18/25, Loss: 0.0113
Epoch 1/500, Iteration 19/25, Loss: 0.0064
Epoch 1/500, Iteration 20/25, Loss: 0.0120
Epoch 1/500, Iteration 21/25, Loss: 0.0126
Epoch 1/500, Iteration 22/25, Loss: 0.0096
Epoch 1/500, Iteration 23/25, Loss: 0.0082
Epoch 1/500, Iterati

C:\Users\sinar\anaconda3\envs\BA\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\sinar\anaconda3\envs\BA\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 2/500, Iteration 4/25, Loss: 0.0110
Epoch 2/500, Iteration 5/25, Loss: 0.0110
Epoch 2/500, Iteration 6/25, Loss: 0.0087
Epoch 2/500, Iteration 7/25, Loss: 0.0099
Epoch 2/500, Iteration 8/25, Loss: 0.0103
Epoch 2/500, Iteration 9/25, Loss: 0.0113
Epoch 2/500, Iteration 10/25, Loss: 0.0095
Epoch 2/500, Iteration 11/25, Loss: 0.0107
Epoch 2/500, Iteration 12/25, Loss: 0.0088
Epoch 2/500, Iteration 13/25, Loss: 0.0119
Epoch 2/500, Iteration 14/25, Loss: 0.0072
Epoch 2/500, Iteration 15/25, Loss: 0.0111
Epoch 2/500, Iteration 16/25, Loss: 0.0137
Epoch 2/500, Iteration 17/25, Loss: 0.0105
Epoch 2/500, Iteration 18/25, Loss: 0.0133
Epoch 2/500, Iteration 19/25, Loss: 0.0074
Epoch 2/500, Iteration 20/25, Loss: 0.0087
Epoch 2/500, Iteration 21/25, Loss: 0.0069
Epoch 2/500, Iteration 22/25, Loss: 0.0069
Epoch 2/500, Iteration 23/25, Loss: 0.0091
Epoch 2/500, Iteration 24/25, Loss: 0.0107
Epoch 2/500, Iteration 25/25, Loss: 0.0089
Train Error: 
 Accuracy: 0.62%, Avg loss: 0.000454 

Test Er

Epoch 10/500, Iteration 8/25, Loss: 0.0131
Epoch 10/500, Iteration 9/25, Loss: 0.0153
Epoch 10/500, Iteration 10/25, Loss: 0.0064
Epoch 10/500, Iteration 11/25, Loss: 0.0094
Epoch 10/500, Iteration 12/25, Loss: 0.0131
Epoch 10/500, Iteration 13/25, Loss: 0.0077
Epoch 10/500, Iteration 14/25, Loss: 0.0116
Epoch 10/500, Iteration 15/25, Loss: 0.0080
Epoch 10/500, Iteration 16/25, Loss: 0.0124
Epoch 10/500, Iteration 17/25, Loss: 0.0101
Epoch 10/500, Iteration 18/25, Loss: 0.0104
Epoch 10/500, Iteration 19/25, Loss: 0.0105
Epoch 10/500, Iteration 20/25, Loss: 0.0123
Epoch 10/500, Iteration 21/25, Loss: 0.0095
Epoch 10/500, Iteration 22/25, Loss: 0.0082
Epoch 10/500, Iteration 23/25, Loss: 0.0093
Epoch 10/500, Iteration 24/25, Loss: 0.0083
Epoch 10/500, Iteration 25/25, Loss: 0.0161
Train Error: 
 Accuracy: 0.62%, Avg loss: 0.000304 

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.001587 

Epoch 11/500, Iteration 1/25, Loss: 0.0134
Epoch 11/500, Iteration 2/25, Loss: 0.0136
Epoch 11/500, Iterat

Epoch 17/500, Iteration 20/25, Loss: 0.0088
Epoch 17/500, Iteration 21/25, Loss: 0.0057
Epoch 17/500, Iteration 22/25, Loss: 0.0115
Epoch 17/500, Iteration 23/25, Loss: 0.0083
Epoch 17/500, Iteration 24/25, Loss: 0.0106
Epoch 17/500, Iteration 25/25, Loss: 0.0070
Train Error: 
 Accuracy: 0.62%, Avg loss: 0.000360 

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.001392 

Epoch 18/500, Iteration 1/25, Loss: 0.0101
Epoch 18/500, Iteration 2/25, Loss: 0.0079
Epoch 18/500, Iteration 3/25, Loss: 0.0104
Epoch 18/500, Iteration 4/25, Loss: 0.0071
Epoch 18/500, Iteration 5/25, Loss: 0.0062
Epoch 18/500, Iteration 6/25, Loss: 0.0105
Epoch 18/500, Iteration 7/25, Loss: 0.0068
Epoch 18/500, Iteration 8/25, Loss: 0.0116
Epoch 18/500, Iteration 9/25, Loss: 0.0092
Epoch 18/500, Iteration 10/25, Loss: 0.0123
Epoch 18/500, Iteration 11/25, Loss: 0.0092
Epoch 18/500, Iteration 12/25, Loss: 0.0102
Epoch 18/500, Iteration 13/25, Loss: 0.0101
Epoch 18/500, Iteration 14/25, Loss: 0.0098
Epoch 18/500, Iteration 1

...

Epoch 464/500, Iteration 21/25, Loss: 0.0110
Epoch 464/500, Iteration 22/25, Loss: 0.0095
Epoch 464/500, Iteration 23/25, Loss: 0.0097
Epoch 464/500, Iteration 24/25, Loss: 0.0073
Epoch 464/500, Iteration 25/25, Loss: 0.0106
Train Error: 
 Accuracy: 0.62%, Avg loss: 0.000400 

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.001256 

Epoch 465/500, Iteration 1/25, Loss: 0.0070
Epoch 465/500, Iteration 2/25, Loss: 0.0159
Epoch 465/500, Iteration 3/25, Loss: 0.0117
Epoch 465/500, Iteration 4/25, Loss: 0.0097
Epoch 465/500, Iteration 5/25, Loss: 0.0061
Epoch 465/500, Iteration 6/25, Loss: 0.0077
Epoch 465/500, Iteration 7/25, Loss: 0.0078
Epoch 465/500, Iteration 8/25, Loss: 0.0117
Epoch 465/500, Iteration 9/25, Loss: 0.0087
Epoch 465/500, Iteration 10/25, Loss: 0.0057
Epoch 465/500, Iteration 11/25, Loss: 0.0087
Epoch 465/500, Iteration 12/25, Loss: 0.0101
Epoch 465/500, Iteration 13/25, Loss: 0.0087
Epoch 465/500, Iteration 14/25, Loss: 0.0081
Epoch 465/500, Iteration 15/25, Loss: 0.0078
Epoch

Epoch 471/500, Iteration 23/25, Loss: 0.0068
Epoch 471/500, Iteration 24/25, Loss: 0.0117
Epoch 471/500, Iteration 25/25, Loss: 0.0101
Train Error: 
 Accuracy: 0.62%, Avg loss: 0.000391 

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.001480 

Epoch 472/500, Iteration 1/25, Loss: 0.0119
Epoch 472/500, Iteration 2/25, Loss: 0.0083
Epoch 472/500, Iteration 3/25, Loss: 0.0088
Epoch 472/500, Iteration 4/25, Loss: 0.0105
Epoch 472/500, Iteration 5/25, Loss: 0.0095
Epoch 472/500, Iteration 6/25, Loss: 0.0105
Epoch 472/500, Iteration 7/25, Loss: 0.0056
Epoch 472/500, Iteration 8/25, Loss: 0.0058
Epoch 472/500, Iteration 9/25, Loss: 0.0097
Epoch 472/500, Iteration 10/25, Loss: 0.0095
Epoch 472/500, Iteration 11/25, Loss: 0.0141
Epoch 472/500, Iteration 12/25, Loss: 0.0077
Epoch 472/500, Iteration 13/25, Loss: 0.0143
Epoch 472/500, Iteration 14/25, Loss: 0.0110
Epoch 472/500, Iteration 15/25, Loss: 0.0097
Epoch 472/500, Iteration 16/25, Loss: 0.0094
Epoch 472/500, Iteration 17/25, Loss: 0.0071
Epoch

Epoch 480/500, Iteration 13/25, Loss: 0.0119
Epoch 480/500, Iteration 14/25, Loss: 0.0077
Epoch 480/500, Iteration 15/25, Loss: 0.0084
Epoch 480/500, Iteration 16/25, Loss: 0.0104
Epoch 480/500, Iteration 17/25, Loss: 0.0087
Epoch 480/500, Iteration 18/25, Loss: 0.0096
Epoch 480/500, Iteration 19/25, Loss: 0.0106
Epoch 480/500, Iteration 20/25, Loss: 0.0158
Epoch 480/500, Iteration 21/25, Loss: 0.0132
Epoch 480/500, Iteration 22/25, Loss: 0.0070
Epoch 480/500, Iteration 23/25, Loss: 0.0121
Epoch 480/500, Iteration 24/25, Loss: 0.0089
Epoch 480/500, Iteration 25/25, Loss: 0.0090
Train Error: 
 Accuracy: 0.62%, Avg loss: 0.000503 

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.001357 

Epoch 481/500, Iteration 1/25, Loss: 0.0115
Epoch 481/500, Iteration 2/25, Loss: 0.0084
Epoch 481/500, Iteration 3/25, Loss: 0.0097
Epoch 481/500, Iteration 4/25, Loss: 0.0090
Epoch 481/500, Iteration 5/25, Loss: 0.0100
Epoch 481/500, Iteration 6/25, Loss: 0.0068
Epoch 481/500, Iteration 7/25, Loss: 0.0070
Epo

Train Error: 
 Accuracy: 0.62%, Avg loss: 0.000516 

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.001300 

Epoch 488/500, Iteration 1/25, Loss: 0.0077
Epoch 488/500, Iteration 2/25, Loss: 0.0104
Epoch 488/500, Iteration 3/25, Loss: 0.0090
Epoch 488/500, Iteration 4/25, Loss: 0.0094
Epoch 488/500, Iteration 5/25, Loss: 0.0094
Epoch 488/500, Iteration 6/25, Loss: 0.0085
Epoch 488/500, Iteration 7/25, Loss: 0.0100
Epoch 488/500, Iteration 8/25, Loss: 0.0135
Epoch 488/500, Iteration 9/25, Loss: 0.0088
Epoch 488/500, Iteration 10/25, Loss: 0.0065
Epoch 488/500, Iteration 11/25, Loss: 0.0087
Epoch 488/500, Iteration 12/25, Loss: 0.0132
Epoch 488/500, Iteration 13/25, Loss: 0.0082
Epoch 488/500, Iteration 14/25, Loss: 0.0124
Epoch 488/500, Iteration 15/25, Loss: 0.0119
Epoch 488/500, Iteration 16/25, Loss: 0.0084
Epoch 488/500, Iteration 17/25, Loss: 0.0132
Epoch 488/500, Iteration 18/25, Loss: 0.0084
Epoch 488/500, Iteration 19/25, Loss: 0.0090
Epoch 488/500, Iteration 20/25, Loss: 0.0077
Epoch

Epoch 495/500, Iteration 11/25, Loss: 0.0087
Epoch 495/500, Iteration 12/25, Loss: 0.0114
Epoch 495/500, Iteration 13/25, Loss: 0.0088
Epoch 495/500, Iteration 14/25, Loss: 0.0086
Epoch 495/500, Iteration 15/25, Loss: 0.0109
Epoch 495/500, Iteration 16/25, Loss: 0.0088
Epoch 495/500, Iteration 17/25, Loss: 0.0065
Epoch 495/500, Iteration 18/25, Loss: 0.0115
Epoch 495/500, Iteration 19/25, Loss: 0.0106
Epoch 495/500, Iteration 20/25, Loss: 0.0090
Epoch 495/500, Iteration 21/25, Loss: 0.0138
Epoch 495/500, Iteration 22/25, Loss: 0.0153
Epoch 495/500, Iteration 23/25, Loss: 0.0099
Epoch 495/500, Iteration 24/25, Loss: 0.0085
Epoch 495/500, Iteration 25/25, Loss: 0.0099
Train Error: 
 Accuracy: 0.62%, Avg loss: 0.000376 

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.001424 

Epoch 496/500, Iteration 1/25, Loss: 0.0118
Epoch 496/500, Iteration 2/25, Loss: 0.0096
Epoch 496/500, Iteration 3/25, Loss: 0.0066
Epoch 496/500, Iteration 4/25, Loss: 0.0096
Epoch 496/500, Iteration 5/25, Loss: 0.0075
E